In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from gensim.models import Word2Vec

In [82]:
df_train= pd.read_csv('train.csv')
df_test=pd.read_csv('inference.csv')

In [26]:
df_train.head(15)

,id,query,label
0,0,شرایط حذف ترم چیه؟,1
1,1,از کجا می تونم با دکتر وحیدی ارتباط برقرار کنم؟,2
2,2,بوفه برداران تا ساعت چند باز است؟,2
3,3,کمترین تعداد واحد چند عدد است؟,1
4,4,سنگ جامد است,5
5,5,سرورای دانشکده مشکل دارن؟,3
6,6,کلاس آزمایشگاه فیزیک در دانشکده خودمان برگزار ...,2
7,7,شرایط حذف پزشکی چیه؟,1
8,8,در شرایطی ساعت و روز کلاسی جابجا می شود؟,1
9,9,سطل آشغال در کلاس 101 وجود ندارد.,4


In [83]:
df_test

,Unnamed: 0,query,label
0,0,آیا درس فلان همیشه توسط یک استاد ارایه میشود؟,1
1,1,آیا میشه درسی از ارشد برداشت؟,1
2,2,لطفاً اختیاری های بهتری ارايه کنید ما انتخابی ...,4
3,3,آیا میتوانیم درس شبکه های عصبی ارشد را برداریم؟,1
4,4,هیچکس در دانشگاه واسخگوی مطالبات دانشجویان نیست,4
5,5,آیا سیاست های جدید دولت بر قیمت نفت تاثیر دارد؟,5
6,6,چرا سرور دانشکده پایین است؟,3
7,7,لطفا به من درباره دانشکده توضیح مختصری بدهید.,2
8,8,لطفا نمره ی پروژه ی درس برنامه نویسی بیشتر شود.,4
9,9,هوا هفته ی آینده بارانی است.,5


In [5]:
df_train['query'].value_counts()

برای ثبت نام باید کجا برم؟                    13
کتابخانه کجاست؟                                7
سایت دانشکده کجاست؟                            5
لابی دانشکده کجاست؟                            4
امروز چندمه؟                                   4
                                              ..
حراست چیه؟                                     1
زمان انتخاب واحد بد موقع است.                  1
عباسپور کجاست؟                                 1
از چه طریق باید نهار روز فروش خریداری کنم؟     1
اسانسور را درست نمیکنید؟                       1
Name: query, Length: 2912, dtype: int64

In [6]:
df_train['label'].value_counts()

1    820
2    712
4    581
5    470
3    465
Name: label, dtype: int64

In [7]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [9]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    punctuationfree="".join([i for i in text if i not in ["،","؟"]])
    return punctuationfree

df_train['query']= df_train['query'].apply(lambda x:remove_punctuation(x))
df_train

,id,query,label
0,0,شرایط حذف ترم چیه,1
1,1,از کجا می تونم با دکتر وحیدی ارتباط برقرار کنم,2
2,2,بوفه برداران تا ساعت چند باز است,2
3,3,کمترین تعداد واحد چند عدد است,1
4,4,سنگ جامد است,5
...,...,...,...
3043,3043,چند تا درس میشه حذف کرد,1
3044,3044,جدید ترین ویرایش کتاب هریس که موجوده چیه,3
3045,3045,شرایط مهمان شدن در دانشکده ما چیست,1
3046,3046,آمفی تئاتر دانشکده کامپیوتر کجاست,2


In [27]:
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

df_train['query']= df_train['query'].apply(lambda x:preprocess(x))
df_train.head(10)

,id,query,label
0,0,شرایط حذف ترم چیه,1
1,1,از کجا می تونم با دکتر وحیدی ارتباط برقرار کنم,2
2,2,بوفه برداران تا ساعت چند باز است,2
3,3,کمترین تعداد واحد چند عدد است,1
4,4,سنگ جامد است,5
5,5,سرورای دانشکده مشکل دارن,3
6,6,کلاس آزمایشگاه فیزیک در دانشکده خودمان برگزار ...,2
7,7,شرایط حذف پزشکی چیه,1
8,8,در شرایطی ساعت و روز کلاسی جابجا می شود,1
9,9,سطل آشغال در کلاس وجود ندارد,4


In [11]:
from hazm import *
from hazm import stopwords_list
stops = stopwords_list()
def stopword(string):
    a= [i for i in string.split() if i not in stops]
    return ' '.join(a)

df_train['query']= df_train['query'].apply(lambda x:stopword(x))
df_train

,id,query,label
0,0,شرایط حذف ترم چیه,1
1,1,کجا تونم دکتر وحیدی ارتباط برقرار,2
2,2,بوفه برداران ساعت,2
3,3,کمترین واحد عدد,1
4,4,سنگ جامد,5
...,...,...,...
3043,3043,درس میشه حذف,1
3044,3044,ترین ویرایش کتاب هریس موجوده چیه,3
3045,3045,شرایط مهمان دانشکده چیست,1
3046,3046,آمفی تئاتر دانشکده کامپیوتر کجاست,2


In [29]:
my_tokenizer = lambda x: word_tokenize(x["query"])
df_train["tokened"] = df_train.apply(my_tokenizer, axis=1)
df_train

,id,query,label,tokened
0,0,شرایط حذف ترم چیه,1,"[شرایط, حذف, ترم, چیه]"
1,1,از کجا می تونم با دکتر وحیدی ارتباط برقرار کنم,2,"[از, کجا, می, تونم, با, دکتر, وحیدی, ارتباط, ب..."
2,2,بوفه برداران تا ساعت چند باز است,2,"[بوفه, برداران, تا, ساعت, چند, باز, است]"
3,3,کمترین تعداد واحد چند عدد است,1,"[کمترین, تعداد, واحد, چند, عدد, است]"
4,4,سنگ جامد است,5,"[سنگ, جامد, است]"
...,...,...,...,...
3043,3043,چند تا درس میشه حذف کرد,1,"[چند, تا, درس, میشه, حذف, کرد]"
3044,3044,جدید ترین ویرایش کتاب هریس که موجوده چیه,3,"[جدید, ترین, ویرایش, کتاب, هریس, که, موجوده, چیه]"
3045,3045,شرایط مهمان شدن در دانشکده ما چیست,1,"[شرایط, مهمان, شدن, در, دانشکده, ما, چیست]"
3046,3046,آمفی تئاتر دانشکده کامپیوتر کجاست,2,"[آمفی, تئاتر, دانشکده, کامپیوتر, کجاست]"


In [38]:
unuqe_tokenze=set()
class_1_count=0;
class_2_count=0;
class_3_count=0;
class_4_count=0;
class_5_count=0;
class_1_sentances=0;
class_2_sentances=0;
class_3_sentances=0;
class_4_sentances=0;
class_5_sentances=0;
tokens_in_each_class = {1: [] ,
                        2: [],
                        3: [],
                        4: [],
                        5: []}
size = len(df_train.index)
for i in range(size):
    line = df_train.iloc[i]
    #tokens_in_each_class[line.label].append(token for token in line.tokened)
    for j in range(len(line.tokened)):
        tokens_in_each_class[line.label].append(line.tokened[j])
        unuqe_tokenze.add(line.tokened[j])
        if line.label==1:
            class_1_count+=1;
        if line.label==2:
            class_2_count+=1;
        if line.label==3:
            class_3_count+=1;
        if line.label==4:
            class_4_count+=1;
        if line.label==5:
            class_5_count+=1;
    if line.label==1:
        class_1_sentances+=1;
    if line.label==2:
        class_2_sentances+=1;
    if line.label==3:
        class_3_sentances+=1;
    if line.label==4:
        class_4_sentances+=1;
    if line.label==5:
        class_5_sentances+=1;

In [39]:
print(len(tokens_in_each_class[1]))
print(len(tokens_in_each_class[2]))
print(len(tokens_in_each_class[3]))
print(len(tokens_in_each_class[4]))
print(len(tokens_in_each_class[5]))
print(class_1_sentances)
print(len(unuqe_tokenze))

5676
3757
3069
4334
2612
820
2604


In [84]:
my_tokenizer = lambda x: word_tokenize(x["query"])
df_test['query']= df_test['query'].apply(lambda x:remove_punctuation(x))
df_test['query']= df_test['query'].apply(lambda x:preprocess(x))
df_test['query']= df_test['query'].apply(lambda x:stopword(x))
df_test["tokened"] = df_test.apply(my_tokenizer, axis=1)
df_test["label"]=0
df_test

,Unnamed: 0,query,label,tokened
0,0,درس فلان توسط استاد ارایه میشود,0,"[درس, فلان, توسط, استاد, ارایه, میشود]"
1,1,میشه درسی ارشد برداشت,0,"[میشه, درسی, ارشد, برداشت]"
2,2,لطفا اختیاری های بهتری ارايه انتخابی نداریم,0,"[لطفا, اختیاری, های, بهتری, ارايه, انتخابی, ند..."
3,3,میتوانیم درس شبکه های عصبی ارشد برداریم,0,"[میتوانیم, درس, شبکه, های, عصبی, ارشد, برداریم]"
4,4,هیچکس دانشگاه واسخگوی مطالبات دانشجویان,0,"[هیچکس, دانشگاه, واسخگوی, مطالبات, دانشجویان]"
5,5,سیاست های دولت قیمت نفت تاثیر,0,"[سیاست, های, دولت, قیمت, نفت, تاثیر]"
6,6,سرور دانشکده پایین,0,"[سرور, دانشکده, پایین]"
7,7,لطفا دانشکده توضیح مختصری بدهید,0,"[لطفا, دانشکده, توضیح, مختصری, بدهید]"
8,8,لطفا نمره ی پروژه ی درس برنامه نویسی,0,"[لطفا, نمره, ی, پروژه, ی, درس, برنامه, نویسی]"
9,9,هوا هفته ی آینده بارانی,0,"[هوا, هفته, ی, آینده, بارانی]"


In [42]:
def counter(word , class_number):
    list_of_tokens = tokens_in_each_class[class_number]
    count=0
    for i in range(len(list_of_tokens)):
        if list_of_tokens[i] == word:
            count+=1
    return count

In [43]:
def number_of_token_in_class(number):
    if number == 1:
        return class_1_count
    if number == 2:
        return class_2_count
    if number == 3:
        return class_3_count
    if number == 4:
        return class_4_count
    if number == 5:
        return class_5_count

In [45]:
number_of_all_classes = class_1_sentances + class_2_sentances + class_3_sentances + class_4_sentances + class_5_sentances
#print(number_of_all_classes)
def bayse(word , class_number):
    count_in_class = counter(word , class_number)
    sorat = count_in_class +1
    makhraj = number_of_token_in_class(class_number) + len(unuqe_tokenze)
    return sorat/makhraj

In [46]:
def nesbat_in_class_be_col(number):
    if number == 1:
        return class_1_sentances/number_of_all_classes
    if number == 2:
        return class_2_sentances/number_of_all_classes
    if number == 3:
        return class_3_sentances/number_of_all_classes
    if number == 4:
        return class_4_sentances/number_of_all_classes
    if number == 5:
        return class_5_sentances/number_of_all_classes

In [85]:
def calculate_bayse(tokens):
    list_of_ehtamalat=[]
    for j in [1,2,3,4,5]:
        ehtemal = 1
        ehteaml= nesbat_in_class_be_col(j)
        for i in range(len(tokens)):
            word = tokens[i]
            ehtemal = ehtemal * bayse(word,j)
        list_of_ehtamalat.append(ehtemal)
    max_ehtemal = max(list_of_ehtamalat)
    #print(max_ehtemal)
    #print(list_of_ehtamalat)
    index = list_of_ehtamalat.index(max_ehtemal)
    return index + 1      

In [86]:
size = len(df_test.index)
list_of_labels=[]
for i in range(size):
    line = df_test.iloc[i]
    #print(line.tokened)
    list_of_labels.append( calculate_bayse(line.tokened))
    #print(calculate_bayse(line.tokened))

df_test["label"]=list_of_labels
df_test

,Unnamed: 0,query,label,tokened
0,0,درس فلان توسط استاد ارایه میشود,4,"[درس, فلان, توسط, استاد, ارایه, میشود]"
1,1,میشه درسی ارشد برداشت,1,"[میشه, درسی, ارشد, برداشت]"
2,2,لطفا اختیاری های بهتری ارايه انتخابی نداریم,4,"[لطفا, اختیاری, های, بهتری, ارايه, انتخابی, ند..."
3,3,میتوانیم درس شبکه های عصبی ارشد برداریم,1,"[میتوانیم, درس, شبکه, های, عصبی, ارشد, برداریم]"
4,4,هیچکس دانشگاه واسخگوی مطالبات دانشجویان,5,"[هیچکس, دانشگاه, واسخگوی, مطالبات, دانشجویان]"
5,5,سیاست های دولت قیمت نفت تاثیر,5,"[سیاست, های, دولت, قیمت, نفت, تاثیر]"
6,6,سرور دانشکده پایین,3,"[سرور, دانشکده, پایین]"
7,7,لطفا دانشکده توضیح مختصری بدهید,4,"[لطفا, دانشکده, توضیح, مختصری, بدهید]"
8,8,لطفا نمره ی پروژه ی درس برنامه نویسی,4,"[لطفا, نمره, ی, پروژه, ی, درس, برنامه, نویسی]"
9,9,هوا هفته ی آینده بارانی,5,"[هوا, هفته, ی, آینده, بارانی]"


In [49]:
#result_df = df_test.get("id","label")
result_df=pd.DataFrame(df_test.get("label"))
#result_df.index.names = ['id']
result_df

#rr = df_test[["id","label"]].copy()
#rr

,label
0,4
1,1
2,4
3,2
4,2
...,...
757,5
758,2
759,1
760,5


In [54]:
result_df.to_csv("result4.csv")
df_train.to_csv("train-after.csv")
df_test.to_csv("test-after.csv")

##### Use second classifier

In [57]:
df_train2= pd.read_csv('train.csv')

In [61]:
df_train2['token']=[word_tokenize(i) for i in df_train2['query']]
model = Word2Vec(df_train['token'],min_count=1)


w2v = dict(zip(model.wv.index_to_key , model.wv.vectors))
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(word2vec.values())))

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [62]:
df_train2

,id,query,label,token
0,0,شرایط حذف ترم چیه؟,1,"[شرایط, حذف, ترم, چیه, ؟]"
1,1,از کجا می تونم با دکتر وحیدی ارتباط برقرار کنم؟,2,"[از, کجا, می, تونم, با, دکتر, وحیدی, ارتباط, ب..."
2,2,بوفه برداران تا ساعت چند باز است؟,2,"[بوفه, برداران, تا, ساعت, چند, باز, است, ؟]"
3,3,کمترین تعداد واحد چند عدد است؟,1,"[کمترین, تعداد, واحد, چند, عدد, است, ؟]"
4,4,سنگ جامد است,5,"[سنگ, جامد, است]"
...,...,...,...,...
3043,3043,چند تا درس میشه حذف کرد,1,"[چند, تا, درس, میشه, حذف, کرد]"
3044,3044,جدید ترین ویرایش کتاب هریس که موجوده چیه؟,3,"[جدید, ترین, ویرایش, کتاب, هریس, که, موجوده, چ..."
3045,3045,شرایط مهمان شدن در دانشکده ما چیست؟,1,"[شرایط, مهمان, شدن, در, دانشکده, ما, چیست, ؟]"
3046,3046,آمفی تئاتر دانشکده کامپیوتر کجاست؟,2,"[آمفی, تئاتر, دانشکده, کامپیوتر, کجاست, ؟]"


In [63]:
X_train, X_val, y_train, y_val = train_test_split(df_train2["query"],
                                                  df_train2["label"],
                                                  test_size=0.2,
                                                  shuffle=True)
X_train_tok= [nltk.word_tokenize(i) for i in X_train]
X_val_tok= [nltk.word_tokenize(i) for i in X_val]


tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_vectors_tfidf = tfidf_vectorizer.transform(X_val)

modelw = MeanEmbeddingVectorizer(w2v)
X_train_vectors_w2v = modelw.transform(X_train_tok)
X_val_vectors_w2v = modelw.transform(X_val_tok)

In [64]:
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_vectors_tfidf, y_train)


y_predict = nb_tfidf.predict(X_val_vectors_tfidf)
y_prob = nb_tfidf.predict_proba(X_val_vectors_tfidf).argmax(axis=1)

 

print(classification_report(y_val,y_predict))


              precision    recall  f1-score   support

           1       0.71      0.94      0.81       165
           2       0.68      0.81      0.74       154
           3       0.87      0.76      0.81        91
           4       0.92      0.87      0.89       104
           5       0.84      0.27      0.41        96

    accuracy                           0.76       610
   macro avg       0.80      0.73      0.73       610
weighted avg       0.78      0.76      0.74       610



In [65]:

lr_w2v=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_w2v.fit(X_train_vectors_w2v, y_train) 


y_predict = lr_w2v.predict(X_val_vectors_w2v)
y_prob = lr_w2v.predict_proba(X_val_vectors_w2v)[:,1]
 

print(classification_report(y_val,y_predict))
print('Confusion Matrix:',confusion_matrix(y_val, y_predict))


              precision    recall  f1-score   support

           1       0.35      0.88      0.50       165
           2       0.49      0.36      0.41       154
           3       0.00      0.00      0.00        91
           4       0.60      0.03      0.06       104
           5       0.39      0.30      0.34        96

    accuracy                           0.38       610
   macro avg       0.36      0.31      0.26       610
weighted avg       0.38      0.38      0.30       610

Confusion Matrix: [[145   9   0   1  10]
 [ 75  55   3   0  21]
 [ 71  15   0   0   5]
 [ 72  19   0   3  10]
 [ 51  15   0   1  29]]


In [70]:
df_test=pd.read_csv('test.csv')  #reading the data
df_test['query'] = df_test['query'].apply(lambda x: preprocess(x))
df_test['query'] = df_test['query'].apply(lambda x: stopword(x))
X_test=df_test['query'] 
X_vector=tfidf_vectorizer.transform(X_test) 
y_predict = lr_tfidf.predict(X_vector)     
y_prob = lr_tfidf.predict_proba(X_vector)[:,1]
df_test['predict_prob']= y_prob
df_test['label']= y_predict
#print(df_test.head())
final=df_test[['label']].reset_index(drop=True)
final.to_csv('w2vResult.csv')

In [79]:
from random import randrange
from math import sqrt
from math import exp
from math import pi

def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split
 

def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0
 

def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores